In [1]:
from transit_parser import *

## NJ Transit 

In [2]:
train_name = './test/' + '2018_01_30_3177'

In [3]:
t = TrainParser(train_name)
times = t.get_stop_times()
rows = t.get_rows(times)
df = t.get_df(rows)
performance = t.join_schedule(df)

In [4]:
performance[['from', 'to','status', 'time', 'expected','train_id']]

,from,to,status,time,expected,train_id
stop_num,,,,,,
1,New York Penn Station,New York Penn Station,Departed,2018-01-30 19:21:12,2018-01-30 19:22:00,3177
2,New York Penn Station,Newark Penn Station,Departed,2018-01-30 19:41:23,2018-01-30 19:38:00,3177
3,Newark Penn Station,Rahway,Departed,2018-01-30 19:53:14,2018-01-30 19:51:00,3177
4,Rahway,Metropark,Departed,2018-01-30 19:58:39,2018-01-30 19:58:00,3177
5,Metropark,Metuchen,Departed,2018-01-30 20:05:17,2018-01-30 20:03:00,3177
6,Metuchen,Edison,Departed,2018-01-30 20:09:15,2018-01-30 20:08:00,3177
7,Edison,New Brunswick,Departed,2018-01-30 20:13:11,2018-01-30 20:13:00,3177
8,New Brunswick,Jersey Avenue,Departed,2018-01-30 20:18:14,2018-01-30 20:16:00,3177


## Amtrak

In [5]:
train_id = './test/' + '2018_01_30_A2117'

In [6]:
a_t = TrainParser(train_id)
a_times = a_t.get_stop_times()
a_rows = a_t.get_rows(a_times)
a_df = a_t.get_df(a_rows)
a_performance = a_t.join_schedule(a_df)

In [7]:
a_performance[['from', 'to','status', 'time', 'expected','train_id']]

,from,to,status,time,expected,train_id
stop_num,,,,,,
1,New York Penn Station,New York Penn Station,Departed,2018-01-30 12:00:06,None,A2117
2,New York Penn Station,Newark Penn Station,Departed,2018-01-30 12:16:03,None,A2117
3,Newark Penn Station,Metropark,Departed,2018-01-30 12:31:10,None,A2117
4,Metropark,Philadelphia,Departed,2018-01-30 13:17:11,None,A2117
